In [1]:
import random
random.seed(42)
import torch
torch.manual_seed(42)

In [2]:
from agents import BasicAttackerAgent, PassiveAgent, WolpertingerDefenderAgent
from evaluation import Evaluator
from game import GameConfig
from models import StateShapeData
from vehicles import CompromiseState, VehicleProvider, Vehicle, Vulnerability

class RubbishVehicleProvider(VehicleProvider):
    def __init__(self) -> None:
        self.max_vulns=1

    def next(self) -> Vehicle:
        return Vehicle(
            10,
            [
                Vulnerability(0.5,2,CompromiseState.NOT_COMPROMISED),
            ]
        )

vehicle_provider=RubbishVehicleProvider()
game_config=GameConfig(
    max_vehicles=30,
    cycle_every=None,
)

attacker = PassiveAgent()
defender=WolpertingerDefenderAgent(
    state_shape_data=StateShapeData(
        num_vehicles=game_config.max_vehicles,
        num_vehicle_features=Vehicle.get_shape()[0],
        num_vulns=vehicle_provider.max_vulns,
        num_vuln_features=Vulnerability.get_shape()[0]
    )
)

engine = Evaluator(
    vehicle_provider=vehicle_provider,
    game_config=game_config,
    num_rounds=1000
)
engine.reset()

c:\Users\TeamD\.conda\envs\subgame\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
vehicle_provider.next()

Vehicle(risk=10, in_platoon=False, vulnerabilities=[Vulnerability(prob=0.5, severity=2, state=<CompromiseState.NOT_COMPROMISED: 1>)])

In [4]:
from training import WolpertingerDefenderAgentTrainer
trainer = WolpertingerDefenderAgentTrainer(
    batch_size=1000,
)

In [6]:
stats_history = trainer.train(
    episodes=100,
    max_steps_per_episode=30,
    defender_agent=defender,
    attacker_agent=attacker,
    warmup=1000,
    evaluator=engine,
)

episode 0 step 0 
episode 0 step 1 
episode 0 step 2 
episode 0 step 3 
episode 0 step 4 
episode 0 step 5 
episode 0 step 6 
episode 0 step 7 
episode 0 step 8 
episode 0 step 9 
episode 0 step 10 
episode 0 step 11 
episode 0 step 12 
episode 0 step 13 
episode 0 step 14 
episode 0 step 15 
episode 0 step 16 
episode 0 step 17 
episode 0 step 18 
episode 0 step 19 
episode 0 step 20 
episode 0 step 21 
episode 0 step 22 
episode 0 step 23 
episode 0 step 24 
episode 0 step 25 
episode 0 step 26 
episode 0 step 27 
episode 0 step 28 
episode 0 step 29 
episode 1 step 0 
episode 1 step 1 
episode 1 step 2 
episode 1 step 3 
episode 1 step 4 
episode 1 step 5 
episode 1 step 6 
episode 1 step 7 
episode 1 step 8 
episode 1 step 9 
episode 1 step 10 
episode 1 step 11 
episode 1 step 12 
episode 1 step 13 
episode 1 step 14 
episode 1 step 15 
episode 1 step 16 
episode 1 step 17 
episode 1 step 18 
episode 1 step 19 
episode 1 step 20 
episode 1 step 21 
episode 1 step 22 
episode 1 ste